In [0]:
from keras.datasets import boston_housing

In [0]:
(train_data, train_targets), (test_data, test_targets) =boston_housing.load_data()

In [16]:
train_data.shape

(404, 13)

In [17]:
test_data.shape

(102, 13)

In [18]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

In [19]:
train_targets.shape

(404,)

In [0]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [0]:
from keras import models
from keras import layers
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
  input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

In [9]:
import numpy as np
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
  [train_data[:i * num_val_samples],
  train_data[(i + 1) * num_val_samples:]],
  axis=0)
  partial_train_targets = np.concatenate(
  [train_targets[:i * num_val_samples],
  train_targets[(i + 1) * num_val_samples:]],
  axis=0)
  model = build_model()
  model.fit(partial_train_data, partial_train_targets,
  epochs=num_epochs, batch_size=1, verbose=1)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

processing fold # 0
Epoch 1/100
303/303 [==============================] - 1s 2ms/step - loss: 202.8727 - mean_absolute_error: 10.7902
Epoch 2/100
303/303 [==============================] - 0s 1ms/step - loss: 30.4580 - mean_absolute_error: 3.8486
Epoch 3/100
303/303 [==============================] - 0s 1ms/step - loss: 21.4460 - mean_absolute_error: 3.1855
Epoch 4/100
303/303 [==============================] - 0s 1ms/step - loss: 18.3308 - mean_absolute_error: 2.9037
Epoch 5/100
303/303 [==============================] - 0s 1ms/step - loss: 16.6767 - mean_absolute_error: 2.6684
Epoch 6/100
303/303 [==============================] - 0s 1ms/step - loss: 15.5555 - mean_absolute_error: 2.5744
Epoch 7/100
303/303 [==============================] - 0s 1ms/step - loss: 14.2659 - mean_absolute_error: 2.5367
Epoch 8/100
303/303 [==============================] - 0s 1ms/step - loss: 13.8770 - mean_absolute_error: 2.4236
Epoch 9/100
303/303 [==============================] - 0s 1ms/step - loss:

In [10]:
all_scores

[2.1038153631852405, 2.1462286533695636, 2.845924991192204, 2.2565925339661024]

In [11]:
np.mean(all_scores)

2.338140385428278

In [12]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
  [train_data[:i * num_val_samples],
  train_data[(i + 1) * num_val_samples:]],
  axis=0)
  partial_train_targets = np.concatenate(
  [train_targets[:i * num_val_samples],
  train_targets[(i + 1) * num_val_samples:]],
  axis=0)
  model = build_model()
  history = model.fit(partial_train_data, partial_train_targets,
  validation_data=(val_data, val_targets),
  epochs=num_epochs, batch_size=1, verbose=1)
  mae_history = history.history['val_mean_absolute_error']
  all_mae_histories.append(mae_history)

processing fold # 0
Train on 303 samples, validate on 101 samples
Epoch 1/500
303/303 [==============================] - 1s 3ms/step - loss: 172.3897 - mean_absolute_error: 10.0023 - val_loss: 30.3994 - val_mean_absolute_error: 3.8365
Epoch 2/500
303/303 [==============================] - 1s 2ms/step - loss: 26.6020 - mean_absolute_error: 3.5059 - val_loss: 20.3485 - val_mean_absolute_error: 2.7861
Epoch 3/500
303/303 [==============================] - 1s 2ms/step - loss: 19.3704 - mean_absolute_error: 2.8774 - val_loss: 17.4677 - val_mean_absolute_error: 2.5921
Epoch 4/500
303/303 [==============================] - 0s 2ms/step - loss: 17.6921 - mean_absolute_error: 2.7148 - val_loss: 14.9366 - val_mean_absolute_error: 2.3443
Epoch 5/500
303/303 [==============================] - 0s 2ms/step - loss: 16.0436 - mean_absolute_error: 2.6406 - val_loss: 14.9908 - val_mean_absolute_error: 2.4067
Epoch 6/500
303/303 [==============================] - 0s 2ms/step - loss: 14.7408 - mean_absolut

KeyboardInterrupt: ignored

In [0]:
average_mae_history = [
np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]


In [0]:
import matplotlib.pyplot as plt
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [0]:
def smooth_curve(points, factor=0.9):
smoothed_points = []
for point in points:
  if smoothed_points:
    previous = smoothed_points[-1]
    smoothed_points.append(previous * factor + point * (1 - factor))
  else:
    smoothed_points.append(point)
  return smoothed_points
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [22]:
model = build_model()
model.fit(train_data, train_targets,
epochs=80, batch_size=16, verbose=1)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

Epoch 1/80
404/404 [==============================] - 0s 1ms/step - loss: 504.7570 - mean_absolute_error: 20.3632
Epoch 2/80
404/404 [==============================] - 0s 128us/step - loss: 332.8925 - mean_absolute_error: 15.7001
Epoch 3/80
404/404 [==============================] - 0s 109us/step - loss: 159.6786 - mean_absolute_error: 9.9906
Epoch 4/80
404/404 [==============================] - 0s 129us/step - loss: 72.4341 - mean_absolute_error: 6.4210
Epoch 5/80
404/404 [==============================] - 0s 116us/step - loss: 45.2385 - mean_absolute_error: 4.9022
Epoch 6/80
404/404 [==============================] - 0s 118us/step - loss: 32.0615 - mean_absolute_error: 4.0968
Epoch 7/80
404/404 [==============================] - 0s 133us/step - loss: 25.6304 - mean_absolute_error: 3.5811
Epoch 8/80
404/404 [==============================] - 0s 126us/step - loss: 21.4952 - mean_absolute_error: 3.3246
Epoch 9/80
404/404 [==============================] - 0s 124us/step - loss: 18.5061 -

In [23]:
 test_mae_score

2.551218528373569

In [0]:
predict_values=model.predict(test_data)

In [25]:
predict_values

array([[ 6.66811  ],
       [17.896114 ],
       [21.75186  ],
       [32.256866 ],
       [25.549417 ],
       [20.881361 ],
       [27.119738 ],
       [22.156546 ],
       [17.75878  ],
       [21.64319  ],
       [19.486979 ],
       [15.51898  ],
       [14.184923 ],
       [43.85086  ],
       [18.3243   ],
       [20.278067 ],
       [25.509676 ],
       [19.432688 ],
       [18.842234 ],
       [25.54867  ],
       [ 9.475884 ],
       [13.284643 ],
       [21.449148 ],
       [15.681063 ],
       [20.562035 ],
       [23.946913 ],
       [27.8584   ],
       [30.373552 ],
       [ 9.826784 ],
       [20.136843 ],
       [19.73804  ],
       [13.707245 ],
       [33.671806 ],
       [23.638172 ],
       [18.229656 ],
       [ 6.1472707],
       [15.552774 ],
       [17.014935 ],
       [17.673319 ],
       [26.743269 ],
       [30.507483 ],
       [28.464071 ],
       [11.847443 ],
       [44.15654  ],
       [28.831446 ],
       [25.634977 ],
       [28.648458 ],
       [17.90

In [26]:
predict_values.shape

(102, 1)

In [27]:
predict_values.ndim

2